In [67]:
import streamlit as st
import folium
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import numpy as np
import tempfile
import os
import json

def load_data():
    df = pd.read_csv("data.csv")
    df["Начало"] = pd.to_datetime(df["Начало"], errors='coerce')
    df["Конец"]  = pd.to_datetime(df["Конец"],  errors='coerce')
    return df

df = load_data()


In [68]:

def calculate_time_spent(df_local, threshold=1e-4):
    df_copy = df_local.copy()
    df_copy.sort_values(by=["Группировка", "Начало"], inplace=True)
    df_copy["dwelling_time"] = 0.0

    for group_name, group_data in df_copy.groupby("Группировка", group_keys=False):
        idx_list = group_data.index.to_list()
        for i in range(len(idx_list) - 1):
            idx_i  = idx_list[i]
            idx_i1 = idx_list[i + 1]
            
            lat_end_i    = group_data.loc[idx_i, "latitude_конеч"]
            lon_end_i    = group_data.loc[idx_i, "longitude_конеч"]
            lat_start_i1 = group_data.loc[idx_i1, "latitude_нач"]
            lon_start_i1 = group_data.loc[idx_i1, "longitude_нач"]
            
            dist = np.sqrt((lat_end_i - lat_start_i1)**2 + (lon_end_i - lon_start_i1)**2)
            t_end   = group_data.loc[idx_i, "Конец"]
            t_start = group_data.loc[idx_i1, "Начало"]
            if pd.notnull(t_end) and pd.notnull(t_start):
                delta_sec = (t_start - t_end).total_seconds()
                if delta_sec > 0:
                    df_copy.at[idx_i, "dwelling_time"] = delta_sec
    
    return df_copy

df_time = calculate_time_spent(df, threshold=1e-4)

# Для HeatMap агрегируем по координатам конечных точек
df_time_sum = df_time.groupby(
    ["latitude_конеч", "longitude_конеч"], dropna=False
)["dwelling_time"].sum().reset_index()



In [69]:
# 2) Слой с HeatMap (по времени пребывания)
heat_data = []
for _, row in df_time_sum.iterrows():
    lat = row["latitude_конеч"]
    lon = row["longitude_конеч"]
    weight = row["dwelling_time"]
    if pd.notnull(lat) and pd.notnull(lon):
        heat_data.append([lat, lon, weight])


In [70]:
df_time[:10]

,Группировка,Начало,latitude_нач,longitude_нач,Конец,latitude_конеч,longitude_конеч,lvl1,lvl2,lvl3,dwelling_time
0,313 камри Сагадиев,2025-03-20 09:55:13,51.111071,71.394976,2025-03-20 09:59:10,51.103013,71.404198,1,1,1,234.0
1,313 камри Сагадиев,2025-03-20 10:03:04,51.103013,71.404198,2025-03-20 10:15:56,51.130757,71.387532,1,1,2,165.0
2,313 камри Сагадиев,2025-03-20 10:18:41,51.130757,71.387532,2025-03-20 10:28:15,51.111071,71.394976,1,1,3,3333.0
3,313 камри Сагадиев,2025-03-20 11:23:48,51.111071,71.394976,2025-03-20 11:33:39,51.128940,71.391480,1,1,4,343.0
4,313 камри Сагадиев,2025-03-20 11:39:22,51.128940,71.391480,2025-03-20 11:40:59,51.130305,71.388520,1,1,5,3984.0
5,313 камри Сагадиев,2025-03-20 12:47:23,51.130757,71.387532,2025-03-20 13:02:39,51.103013,71.404198,1,1,6,2115.0
6,313 камри Сагадиев,2025-03-20 13:37:54,51.103013,71.404198,2025-03-20 14:17:48,51.201770,71.484700,1,1,7,5612.0
7,313 камри Сагадиев,2025-03-20 15:51:20,51.201770,71.484700,2025-03-20 16:40:56,51.103013,71.404198,1,1,8,670.0
8,313 камри Сагадиев,2025-03-20 16:52:06,51.103013,71.404198,2025-03-20 16:56:48,51.111071,71.394976,1,1,9,692.0
9,313 камри Сагадиев,2025-03-20 17:08:20,51.111071,71.394976,2025-03-20 17:16:53,51.111774,71.417765,1,1,10,737.0


In [71]:
df_time[df_time["dwelling_time"]  <= 0].copy()

,Группировка,Начало,latitude_нач,longitude_нач,Конец,latitude_конеч,longitude_конеч,lvl1,lvl2,lvl3,dwelling_time
130,313 камри Сагадиев,2025-03-31 10:15:33,51.130757,71.387532,2025-03-31 10:21:59,51.116295,71.393545,1,12,4,0.0
350,314 камри Нурияхметов,2025-03-31 09:51:49,51.143920,71.361040,2025-03-31 10:06:12,51.113200,71.399990,2,12,6,0.0
431,348 CR Сейтмухамбетов,2025-03-31 08:42:41,51.651790,72.141350,2025-03-31 10:11:15,51.284120,72.829280,3,10,3,0.0
492,352 Балгожа,2025-03-31 09:40:20,53.222870,63.659980,2025-03-31 10:19:53,53.486850,64.035870,4,12,3,0.0
587,358 Мамбетов,2025-03-31 09:56:55,52.905660,70.189340,2025-03-31 10:14:34,52.948020,70.423737,6,12,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7300,974 Лубянецкий,2025-03-31 08:50:49,53.230752,63.662312,2025-03-31 09:10:07,53.210756,63.598391,98,11,1,0.0
7368,986 Авижич,2025-03-30 11:04:37,53.182630,63.588940,2025-03-30 11:08:50,53.182585,63.601305,99,9,5,0.0
7468,997 Кализатов,2025-03-31 08:41:38,53.294943,69.394117,2025-03-31 10:18:28,52.411450,68.625260,100,11,5,0.0
7546,Дауленов 357,2025-03-31 09:00:35,52.275267,76.974713,2025-03-31 09:15:34,52.294723,76.928187,101,12,2,0.0


In [72]:
# Создаем DataFrame только с событиями, где dwelling_time > 0
df_dwell_events = df_time[df_time["dwelling_time"] > 0].copy()
df_dwell_events["Прибытие"] = df_dwell_events["Конец"] + pd.to_timedelta(df_dwell_events["dwelling_time"], unit="s")
df_dwell_events["Конец_str"] = df_dwell_events["Конец"].dt.strftime("%Y-%m-%d %H:%M:%S")
df_dwell_events["Прибытие_str"] = df_dwell_events["Прибытие"].dt.strftime("%Y-%m-%d %H:%M:%S")


In [73]:
# Создаем DataFrame только с событиями, где dwelling_time > 0
df_dwell_events = df_time[df_time["dwelling_time"] > 0].copy()
df_dwell_events["Прибытие"] = df_dwell_events["Конец"] + pd.to_timedelta(df_dwell_events["dwelling_time"], unit="s")
df_dwell_events["Конец_str"] = df_dwell_events["Конец"].dt.strftime("%Y-%m-%d %H:%M:%S")
df_dwell_events["Прибытие_str"] = df_dwell_events["Прибытие"].dt.strftime("%Y-%m-%d %H:%M:%S")


grouped_events = df_dwell_events.groupby(["latitude_конеч", "longitude_конеч"])
popup_texts = {}
popup_texts_1 = []  # Здесь будем сохранять информацию по каждому событию отдельно
agent_tooltips = {}

for (lat, lon), group in grouped_events:
    # Собираем уникальные имена агентов в этой точке
    agents = group["Группировка"].unique()
    agent_names = ", ".join(agents)
    
    # Формируем popup-текст для этой группы (если нужно для отображения на карте)
    lines = [f"Агент: {agent_names}"]
    
    for _, row in group.iterrows():
        # Добавляем строку с временем прибытия и отъезда для popup
        lines.append(f"Прибытие: {row['Прибытие_str']}<br>Отъезд: {row['Конец_str']}")
        # Сохраняем отдельное событие: агент, широта, долгота, время прибытия, время отъезда
        popup_texts_1.append([agent_names, lat, lon, row['Прибытие_str'], row['Конец_str']])
    
    popup_text = "<hr>".join(lines)
    popup_texts[(lat, lon)] = popup_text
    agent_tooltips[(lat, lon)] = agent_names


In [37]:
df_time_sum

,latitude_конеч,longitude_конеч,dwelling_time
0,48.362385,85.607955,69380.0
1,48.692750,83.431360,272.0
2,48.743520,82.692510,2733.0
3,48.754310,82.384610,1412.0
4,48.755350,82.379780,632.0
...,...,...,...
4740,54.944496,69.180802,105654.0
4741,54.951710,69.236820,512.0
4742,54.952020,68.526170,1172.0
4743,54.952340,69.140730,516.0


In [29]:
grouped_events.size()

latitude_конеч  longitude_конеч
48.362385       85.607955          1
48.692750       83.431360          1
48.743520       82.692510          1
48.754310       82.384610          1
48.755350       82.379780          1
                                  ..
54.944496       69.180802          9
54.951710       69.236820          1
54.952020       68.526170          1
54.952340       69.140730          1
54.953190       69.243690          1
Length: 4297, dtype: int64

In [28]:
df_time

,Группировка,Начало,latitude_нач,longitude_нач,Конец,latitude_конеч,longitude_конеч,lvl1,lvl2,lvl3,dwelling_time
0,313 камри Сагадиев,2025-03-20 09:55:13,51.111071,71.394976,2025-03-20 09:59:10,51.103013,71.404198,1,1,1,234.0
1,313 камри Сагадиев,2025-03-20 10:03:04,51.103013,71.404198,2025-03-20 10:15:56,51.130757,71.387532,1,1,2,165.0
2,313 камри Сагадиев,2025-03-20 10:18:41,51.130757,71.387532,2025-03-20 10:28:15,51.111071,71.394976,1,1,3,3333.0
3,313 камри Сагадиев,2025-03-20 11:23:48,51.111071,71.394976,2025-03-20 11:33:39,51.128940,71.391480,1,1,4,343.0
4,313 камри Сагадиев,2025-03-20 11:39:22,51.128940,71.391480,2025-03-20 11:40:59,51.130305,71.388520,1,1,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7630,Джексембаев 353,2025-03-31 08:21:54,49.906150,82.619970,2025-03-31 09:11:01,50.026680,82.495940,102,12,1,152.0
7631,Джексембаев 353,2025-03-31 09:13:33,50.026680,82.495940,2025-03-31 09:26:56,50.076130,82.383270,102,12,2,152.0
7632,Джексембаев 353,2025-03-31 09:29:28,50.076130,82.383270,2025-03-31 09:31:40,50.077467,82.386153,102,12,3,1178.0
7633,Джексембаев 353,2025-03-31 09:51:18,50.077467,82.386153,2025-03-31 09:55:02,50.073113,82.381260,102,12,4,213.0


In [27]:
pd.DataFrame(popup_texts_1)

,0,1,2,3,4
0,459 Жаирбаев,48.362385,85.607955,2025-03-30 07:14:24,2025-03-29 11:58:04
1,459 Жаирбаев,48.692750,83.431360,2025-03-30 09:42:47,2025-03-30 09:38:15
2,Джексембаев 353,48.743520,82.692510,2025-03-27 09:48:02,2025-03-27 09:02:29
3,Джексембаев 353,48.754310,82.384610,2025-03-27 10:50:12,2025-03-27 10:26:40
4,Джексембаев 353,48.755350,82.379780,2025-03-27 10:23:55,2025-03-27 10:13:23
...,...,...,...,...,...
7032,460 Егоров,54.944496,69.180802,2025-03-30 14:42:11,2025-03-30 12:25:39
7033,536 Гаврилюк,54.951710,69.236820,2025-03-28 17:11:39,2025-03-28 17:03:07
7034,452 CR Аскеров,54.952020,68.526170,2025-03-31 10:19:03,2025-03-31 09:59:31
7035,431 Шадрин,54.952340,69.140730,2025-03-28 19:52:17,2025-03-28 19:43:41


In [14]:
pd.DataFrame(popup_texts)

ValueError: If using all scalar values, you must pass an index

In [ ]:
df_2 = pd.DataFrame(heat_data)


,0,1,2
0,48.743520,82.692510,2733.0
1,48.754310,82.384610,1412.0
2,48.755350,82.379780,632.0
3,48.843560,82.747510,1893.0
4,48.869510,83.254280,334.0
...,...,...,...
4792,54.944400,69.197570,1112.0
4793,54.944493,69.180794,89449.0
4794,54.953190,69.243690,2732.0
4795,54.962000,69.037540,1712.0
